# Regression task

In this notebook, we will:
- #### Simulate daily streamflow ($Q$) in a mountain using precipitation, snow, and temperature data
- First, necessary modules are imported.
- Then, the data is loaded and inspected.
- After that, ML starts...
- Making the pipeline.
- Using GridSearch.
- Checking results.

#### Make sure to:
- Get a notion of *how your ML model works*.
- *Select hyperparameters* to optimize.
- Execute *GridSearch*.
- Check model, particularly *test results*.
- Is the methodology *valid*?

#### About the models:
- Linear (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- Polynomial (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)
- Multilayer Perceptron (ANN) (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)
- Random Forests (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
- Support-vector regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

## Load data

Let's load and look at the data using the *pandas* module (`pd.read_csv()`).
Also, we separate the test data to use at the end.


In [ ]:
# This is all is takes to load it!
data = pd.read_csv('regression_data.csv', index_col=0, date_format='%m/%d/%Y')

# To split the data by dates

X = data.loc[data.index<'2021-01-01', :].iloc[:, 1:]
y = data.loc[data.index<'2021-01-01', :].iloc[:, 0]
X_test = data.loc[data.index>='2021-01-01', :].iloc[:, 1:]
y_test = data.loc[data.index>='2021-01-01', :].iloc[:, 0]

# To print
data

## How does it look?

Now we use pandas (`data.plot()`) to create a few plots.  
Its works with `matplotlib`, similarly to `plt.plot()`.

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
axs = axs.flatten()

# Plot 1: Q (Discharge)
data.loc[:, ['Q']].plot(color='steelblue', label='Q [m3/s]', ax=axs[0])
axs[0].set_title('Discharge (Q) [m3/s]')
axs[0].set_xlabel('Date')
axs[0].set_ylabel('Q [m3/s]')
axs[0].legend()
axs[0].grid(True)

# Plot 2: All Precipitations
# (Assuming the base precipitation columns; adjust if you want to include lag or sum columns)
precip_cols = ['Precipitation__1', 'Precipitation__2', 'Precipitation__3',
              'Precipitation_lag1__1', 'Precipitation_lag1__2', 'Precipitation_lag1__3',
              'Precipitation_lag2__1', 'Precipitation_lag2__2', 'Precipitation_lag2__3',
              'Precipitation_lag3__1', 'Precipitation_lag3__2', 'Precipitation_lag3__3',
              'Precipitation_sum5__1', 'Precipitation_sum5__2', 'Precipitation_sum5__3',
              ]
data.loc[:, precip_cols].plot(ax=axs[1])
axs[1].set_title('Precipitations [mm/day]')
axs[1].set_xlabel('Date')
axs[1].set_ylabel('P [mm/day]')
axs[1].legend()
axs[1].grid(True)

# Plot 3: All Snows
snow_cols = ['Snow__1', 'Snow__2', 'Snow__3']
data.loc[:, snow_cols].plot(ax=axs[2])
axs[2].set_title('Snow water equivalent [m]')
axs[2].set_xlabel('Date')
axs[2].set_ylabel('SWE [m]')
axs[2].legend()
axs[2].grid(True)

# Plot 4: All Temperatures
temp_cols = ['Temperature__1', 'Temperature__2', 'Temperature__3']
data.loc[:, temp_cols].plot(ax=axs[3])
axs[3].set_title('Temperature [C]')
axs[3].set_xlabel('Date')
axs[3].set_ylabel('T [C]')
axs[3].legend()
axs[3].grid(True)

plt.tight_layout()
plt.show()

## Hyperparameter tuning with grid search

Now we perform a grid search with k-fold cross-validation to tune hyperparameters.  
Make sure you choose the correct ones. Check the docs and think about the model.  
Use only a few combinations to keep the grid search fast.  


In [ ]:
# Create a pipeline (using placeholders for hyperparameters to be tuned)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('regressor', # YOUR REGRESSOR HERE),  
])

# Define the parameter grid for tuning
param_grid = {
    #'pca__n_components': [2, 4, 10], # Try different dimensionalities
    # HYPERPARAMETERS HERE
}

# Set up k-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Create and run the GridSearchCV (MAE scoring is used)
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_absolute_error', verbose=2)
grid_search.fit(X, y)

# Print the best parameters
print("Best parameters found:")
print(grid_search.best_params_)

## Evaluate test data

Let's see how it works!

In [ ]:
# Get the retrained best estimator
trained_model = grid_search.best_estimator_

# Or, if you wish to train it with custom parameters...
#trained_model = pipeline.set_params(pca__n_components=10).fit(X,y)

y_pred = trained_model.predict(X_test)

# Plot the true signal, training data, and multiple predicted signals.
plt.figure(figsize=(10, 6))
# Plot the true signal line (extrapolated)
plt.plot(y_test.index, y_test.values, label="True streamflow", color="black", linewidth=2)
plt.plot(y_test.index, y_pred, label="Predicted streamflow", color="steelblue", linestyle='--', linewidth=2)

plt.xlabel('Date')
plt.ylabel('Discharge (Q) [m3/s]')
plt.title('Your prediction on the test group')
plt.legend()
plt.tight_layout()
plt.show()

mse = mean_squared_error(y_test.values, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse, '[m3/s]')